In [1]:
import threading
import requests
import time

BASE_URL = "http://localhost:8081"  # Replace with your actual API endpoint

import concurrent_test_cases.utils as utils

import concurrent_test_cases.wallet as wallet
 
user_id = 1001
product_id = 101
initial_stock = 10
product_price = 55000
thread_count = 15  # More than available stock to test rollback


In [2]:
def place_order():
    order_data = {
        "user_id": user_id,
        "items": [{"product_id": product_id, "quantity": 1}]
    }

    response = requests.post(f"{BASE_URL}/orders", json=order_data)

    print(f"Response: {response.status_code}, {response.text}")
    return response.status_code

In [7]:
USR_URL = "http://localhost:8080/users"

In [8]:
def add_user(id,  name, email):
    user_data = {
        "id": id,
        "name": name,
        "email": email
    }

    response = requests.post(f"{USR_URL}", json=user_data)

    print(f"Response: {response.status_code}, {response.text}")
    return response.status_code



In [ ]:
add_user(user_id, "John Doe", "jhn@exp.com")

Response: 201, {"id":1001,"name":"John Doe","email":"jhn@exp.com","discount_availed":false}


201

In [11]:
requests.get(f"{BASE_URL}/products/{product_id}").json()

{'id': 101,
 'name': 'Laptop Pro 1',
 'description': 'Powerful laptop',
 'price': 55000,
 'stock_quantity': 10}